# Running the Report Generation Agent

This notebook runs the **Report Generation Agent** for **single-table relational
data source** as a Gradio Demo UI and evaluations with [Langfuse](https://langfuse.com/).

The Report Generation Agent Gradio Demo will provide an UI to read **user queries in natural language**
and proceed to **make SQL queries** to the database in order to produce the data for
the report. At the end, the Agent will provide a **downloadable link** to the report as
an `.xlsx` file.

This example also provides agent monitoring and evaluations using Langfuse.

## Setting up

The code below sets the notebook default folder, sets the default constants and checks the presence of the environment variables.

The environment variables can be set in the `.env` file in the root folder of the project.

In [ ]:
import os
from pathlib import Path

from aieng.agent_evals.async_client_manager import AsyncClientManager


# Setting the notebook directory to the project's root folder
if Path("").absolute().name == "eval-agents":
    print(f"Notebook path is already the root path: {Path('').absolute()}")
else:
    os.chdir(Path("").absolute().parent.parent)
    print(f"The notebook path has been set to: {Path('').absolute()}")

client_manager = AsyncClientManager.get_instance()
assert client_manager.configs.report_generation_db.database, (
    "[ERROR] The database path is not set! Please configure the REPORT_GENERATION_DB__DATABASE environment variable."
)
assert client_manager.configs.langfuse_secret_key, (
    "[ERROR] The Langfuse secret key is not set! Please configure the LANGFUSE_SECRET_KEY environment variable."
)
assert client_manager.configs.langfuse_public_key, (
    "[ERROR] The Langfuse public key is not set! Please configure the LANGFUSE_PUBLIC_KEY environment variable."
)
assert client_manager.configs.langfuse_host, (
    "[ERROR] The Langfuse base URL is not set! Please configure the LANGFUSE_BASE_URL environment variable."
)

from implementations.report_generation.demo import start_gradio_app  # noqa: E402


print("All environment variables have been set.")

## Running the Demo UI

The code below will start the **Gradio Demo UI**, which will run embedded in this notebook (made possible by the `enable_public_link=True` parameter). If you so choose, you can also run the UI in your browser using the links provided in the outputs.

The UI will display a few **pre-defined input options** to test the agent and also a text box so you can **make your own queries**. The Agent will try to complete the request by making **queries against the database** configured in the first notebook. At the end, it will **produce a report** that it will make available as a download link at the end of the agent's execution.

The UI will display the **agents thoughts** and **tool calls** in order to facilitate debugging.

The agent **traces will be sent** to [Langfuse](https://us.cloud.langfuse.com/). To configure the langfuse connection, you can edit the following environment variables in your `.env` file:

```python
# Secret and public keys can be generated by the Langfuse web UI
LANGFUSE_SECRET_KEY="sk-lf-..."
LANGFUSE_PUBLIC_KEY="pk-lf-..."
LANGFUSE_BASE_URL="https://us.cloud.langfuse.com"

# Input the name of your langfuse project on the first variable
# The second variable is used to configure where the output reports are going to be saved
# Defaults are in implementations/report_generation/env_vars.py)
REPORT_GENERATION_LANGFUSE_PROJECT_NAME="..."
REPORT_GENERATION_OUTPUT_PATH="..."
```

Run the cell below to see the Report Generation Agent in action.

In [ ]:
await start_gradio_app(
    enable_trace=True,
    enable_public_link=True,
)

## Agent Configuration

This agent is a **text-to-SQL** agent that will convert natural language into one or multiple queries to a DB in order to produce the result that has been requested.

This agent in particular has **no knowledge of the DB schema**, although it is possible to do so with some more advanced techniques. This allows the agent to be **flexible** to any database it may encounter. However, it is **easier** to have it working with a **single-table DB** as opposed to a multi-table DB.

It is configured with **three tools** (as per `aieng.agent_evals.evaluation.report_generation.agent.get_report_generation_agent`):
- `db_manager.report_generation_db().get_schema_info`: to retrieve the **DB schema** so the agent knows how to perform the queries.
- `db_manager.report_generation_db().execute`: to execute any **read-only SQL queries**.
- `report_file_writer.write_xlsx`: a function that receives the **report data** as an array, writes the array to a `.xlsx` file and returns a **downloadable Gradio link** to the file.

The Agent will know how to use those tools given the **instructions** it has been given, along with the user input. Below are the instructions for this agent in specific (as per `aieng.agent_evals.evaluation.report_generation.prompts`:
```python
MAIN_AGENT_INSTRUCTIONS = """\
Perform the task using the SQLite database tool. \
EACH TIME before invoking the function, you must explain your reasons for doing so. \
If the SQL query did not return intended results, try again. \
For best performance, divide complex queries into simpler sub-queries. \
Do not make up information. \
When the report is done, use the report file writer tool to write it to a file. \
Make sure the "write_xlsx" tool is called so it generates the report file. \
At the end, provide the report file as a downloadable hyperlink to the user. \
Make sure the link can be clicked on by the user.
"""
```

## Online Evaluations

The Agent will be sending online evaluation metrics to Langfuse along with the traces. These metrics aim to **simulate** how an evaluation of a **production agentic system** would work for this use case.

Those metrics are:
- A check of the **total token spent** to complete the request. If it is below the threshold of 15k tokens it will send a score of 1, or a score of 0 otherwise.
- A check of the **total time spent** to complete the request. If it is below the threshold of 60 seconds it will send a score of 1, or a score of 0 otherwise.
- At the end of the Agent run, the UI will display thumbs up/thumbs down buttons so the **users can send feedback** on the output of the agent.

The calculation for the first two metrics are triggered by a **callback function** that runs at the end of the agent run, while the third one is triggered asynchronously by an **user UI action**.

Those metrics will be available in Langfuse in two different ways:
- **Aggregated:** Summary of those metrics will be displayed on the dashboards page
- **Individualized:** The evaluation scores are available on each trace with additional details